In [8]:
# Set up
import numpy as np
import pandas as pd
import seaborn as sns # for visualiation
import altair as alt
alt.renderers.enable('notebook') # enable altair rendering
from scipy.stats import ttest_ind # t-tests
import statsmodels.formula.api as smf # linear modeling
import statsmodels.api as sm
import matplotlib.pyplot as plt # plotting
import matplotlib
from sklearn import metrics
matplotlib.style.use('ggplot')
%matplotlib inline

In [9]:
listings_df = pd.read_csv('./data/listings.csv')
bookings_df = pd.read_csv('./data/calendar.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
# The columns we have to work with for listings (house details)
listings_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms',

In [11]:
# Looking at things related to the pricing of the house
pricing_df = listings_df[['id','price', 'weekly_price', 'monthly_price']]
pricing_df

,id,price,weekly_price,monthly_price
0,2318,$296.00,NaN,NaN
1,4291,$82.00,$525.00,"$1,925.00"
2,5682,$48.00,$400.00,"$1,250.00"
3,6606,$90.00,$670.00,NaN
4,9419,$65.00,$580.00,"$1,500.00"
5,9460,$80.00,NaN,NaN
6,9531,$165.00,"$1,075.00","$3,000.00"
7,9534,$125.00,$825.00,"$2,500.00"
8,9596,$120.00,$725.00,NaN
9,9909,$125.00,$945.00,"$3,200.00"


## Observations:

There are some houses that don't have a weekly or monthy rate. Further research shows that "price" indicates the _price per night_. This can be multiplied with how often the house is booked to show how much monthly income the owners are making from the home.

In [12]:
# Looking at columns related to booking the homes
bookings_df.columns

Index(['listing_id', 'date', 'available', 'price'], dtype='object')

In [13]:
bookings_df

,listing_id,date,available,price
0,158345,2019-09-13,t,$55.00
1,158345,2019-09-12,t,$55.00
2,158345,2019-09-11,t,$55.00
3,158345,2019-09-10,t,$55.00
4,158345,2019-09-09,t,$55.00
5,158345,2019-09-08,t,$55.00
6,158345,2019-09-07,t,$55.00
7,158345,2019-09-06,t,$55.00
8,158345,2019-09-05,t,$55.00
9,158345,2019-09-04,t,$55.00


## Observations: 

"True" in the available column indicates that the house can be rented out. We want to know monthy income so we might want to only look at where the "available" column is _false_. 

In [14]:
# Looking at the listings where it's false

booked_houses = bookings_df[bookings_df['available'] == 'f']

In [15]:
booked_houses

,listing_id,date,available,price
52,413733,2019-08-01,f,NaN
53,413733,2019-07-31,f,NaN
54,413733,2019-07-30,f,NaN
55,413733,2019-07-29,f,NaN
56,413733,2019-07-28,f,NaN
57,413733,2019-07-27,f,NaN
58,413733,2019-07-26,f,NaN
59,413733,2019-07-25,f,NaN
60,413733,2019-07-24,f,NaN
61,413733,2019-07-23,f,NaN


## Observations:

Many of the houses that are booked don't have a listed price. This may indicate that we need to combine the dataframes together in order to get a price for the listings.

This, combined with the fact that not all houses have weekly and monthly prices may lead us to make estimates based solely off of the `price` column.

## Data preparation

To explore the data further, our columns of interest need to be created. This includes: `estimated_income_per_month`, `estimated_occupancy`, and `bookings_per_month`.


Here is a picture of the equations the website uses to calculate this information (we should also justify this during our paper as to why we chose these equations / why they represent what we're measuring):

![equations](./images/equations.png)

Here are constants (the variables in all caps) within the equations that were found on the page:

![constants](./images/constants.png)

## Equations for calculation:

- `bookings per month`: number of reviews per month divided by the review rate, rounded to one decimal place.
- `estimated occupancy`: the minimum between bookings per month * the max between the minimum nights and the average nights divided by 30 or the maximum occupancy rounded to 3 decimal places
- `income per month`: estimated occupancy times 30 times the price per night, rounded to the nearest whole number.

In [16]:
# Declare the constants

REVIEW_RATE = 0.3
AVERAGE_NIGHTS = 5
MAXIMUM_OCCUPANCY = 0.7

## Calculate the bookings per month
reviews_per_month = listings_df[['reviews_per_month']]

## Create a copy of the original dataframe to modify
listings_with_income = listings_df.copy()

# Bookings per month
listings_with_income['bookings_per_month'] = reviews_per_month / REVIEW_RATE

In [17]:
# Estimated occupancy

# Get minumum number of nights
minimum_nights = listings_df[['minimum_nights']]

# Create array to hold max number of nights stayed
max_nights_stayed = []

# For each row, choose the max between the minimum number of nights and the average number of nights
for index, min_nights in minimum_nights.iterrows():
    max_nights_stayed.append(max(float(min_nights), AVERAGE_NIGHTS))
    
# Calculate average number of stays per day to multiply by bookings per month to get number of bookings per month
average_stays_per_day = [max_nights / 30 for max_nights in max_nights_stayed]

# Holds average stays per month
average_stays_per_month = listings_with_income['bookings_per_month'] * average_stays_per_day

# Final value for estimated occupancy
estimated_occupancy = []
for average_stay_per_month in average_stays_per_month:
    estimated_occupancy.append(min(average_stay_per_month, MAXIMUM_OCCUPANCY))

# Add the values to the dataframe
listings_with_income['estimated_occupancy'] = estimated_occupancy

In [18]:
# Estimated income per month
prices = listings_with_income['price']
formatted_prices = []

for price in prices:
    formatted_price = price.replace("$", "")
    formatted_price = formatted_price.replace(",", "")
    formatted_prices.append(formatted_price)

listings_with_income['price'] = formatted_prices

listings_with_income.price = listings_with_income.price.astype(float)

listings_with_income['income_per_month'] = round(listings_with_income['estimated_occupancy'] * 30 * listings_with_income['price'])